In [1]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# FILES AUDITORIA

In [2]:
url = r'C:\Users\Cristian.Aparicio\Documents\POLIZAS NETSUITE\NETSUITE\2022\06. JUNIO\Revision_poliza_ns_junio_2022_V2.xlsx'

In [3]:
pol_netsuite = pd.ExcelFile(url)
pol_netsuite

In [4]:
pol_netsuite.sheet_names

['2022.06.30',
 '2022.06.29',
 '2022.06.28',
 '2022.06.27',
 '2022.06.26',
 '2022.06.25',
 '2022.06.24',
 '2022.06.23',
 '2022.06.22',
 '2022.06.21',
 '2022.06.20',
 '2022.06.19',
 '2022.06.18',
 '2022.06.17',
 '2022.06.16',
 '2022.06.15',
 '2022.06.14',
 '2022.06.13',
 '2022.06.12',
 '2022.06.11',
 '2022.06.10',
 '2022.06.09',
 '2022.06.08',
 '2022.06.07',
 '2022.06.06',
 '2022.06.05',
 '2022.06.04',
 '2022.06.03',
 '2022.06.02',
 '2022.06.01']

In [5]:
# df1 = auditoria.parse(('Dep 05-08-2022 Afil 7927518 '),converters={'ORDER_ID':str,'COD_AUTORIZACION': str,'BIN':str,'ULTIMOS_4':str})
# df1.head()

In [6]:
# auditoria.head()

In [7]:
all_sheets = pd.read_excel(url, sheet_name=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29])

In [8]:
all_sheets.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [9]:
audit_1 = pd.concat(all_sheets).reset_index()
audit_1.columns = audit_1.columns.str.strip().str.lower()
audit_1.head()

,level_0,level_1,cuenta,débito,crédito,nota,pbc account type,manual,diff
0,0,0,219-05-00 settlement process CXP : CxP Loans T...,NaN,"1,255,648.14",Adj Loans Lenders,Cuentas por pagar,"1,255,648.14",0.00
1,0,1,219-01-00 settlement process CXP : Depósitos p...,NaN,"2,900.00",Adjustments Cancelaciones,Cuentas por pagar,"2,900.00",0.00
2,0,2,219-01-00 settlement process CXP : Depósitos p...,104.40,NaN,Adjustments Cancelaciones Fees,Cuentas por pagar,104.40,0.00
3,0,3,219-01-00 settlement process CXP : Depósitos p...,16.70,NaN,Adjustments Cancelaciones Taxes,Cuentas por pagar,16.70,0.00
4,0,4,192-05-00 Settlement process CXC : CXC Contrac...,NaN,"6,831.88",Cobranza,Cuentas por cobrar,"6,831.88",0.00


In [10]:
audit_1.dtypes

level_0               int64
level_1               int64
cuenta               object
débito              float64
crédito             float64
nota                 object
pbc account type     object
manual              float64
diff                float64
dtype: object

In [11]:
audit_1.shape

(960, 9)

In [12]:
audit_final = audit_1[['cuenta','débito','crédito','nota','pbc account type']]
audit_final['débito'].fillna(0,inplace=True)
audit_final['crédito'].fillna(0,inplace=True)
audit_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1564\1526520708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  audit_final['débito'].fillna(0,inplace=True)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1564\1526520708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  audit_final['crédito'].fillna(0,inplace=True)


,cuenta,débito,crédito,nota,pbc account type
0,219-05-00 settlement process CXP : CxP Loans T...,0.00,"1,255,648.14",Adj Loans Lenders,Cuentas por pagar
1,219-01-00 settlement process CXP : Depósitos p...,0.00,"2,900.00",Adjustments Cancelaciones,Cuentas por pagar
2,219-01-00 settlement process CXP : Depósitos p...,104.40,0.00,Adjustments Cancelaciones Fees,Cuentas por pagar
3,219-01-00 settlement process CXP : Depósitos p...,16.70,0.00,Adjustments Cancelaciones Taxes,Cuentas por pagar
4,192-05-00 Settlement process CXC : CXC Contrac...,0.00,"6,831.88",Cobranza,Cuentas por cobrar


In [13]:
audit_final.shape

(960, 5)

### GROUP ALL JUNIO 2022

In [14]:
group_poliza = audit_final.groupby(['nota','cuenta']).sum().reset_index()
group_poliza.head()

,nota,cuenta,débito,crédito
0,-1.92,DIFERENCIA,0.00,0.00
1,-0.00,DIFERENCIA,0.00,0.00
2,0,DIFERENCIA,0.00,0.00
3,0.95,DIFERENCIA,0.00,0.00
4,291.31,DIFERENCIA,0.00,0.00


In [15]:
group_poliza.shape

(92, 4)

### GROUP 30-31 JULIO 2022

In [ ]:
sheets_30_31 = pd.read_excel(url, sheet_name=[0,1])

In [ ]:
sheets_30_31.keys()

In [ ]:
audit_2 = pd.concat(sheets_30_31).reset_index()
audit_2.columns = audit_2.columns.str.strip().str.lower()
audit_2.head()

In [ ]:
audit_2.shape

In [ ]:
audit_2.dtypes

In [ ]:
audit_final_30_31 = audit_2[['cuenta','débito','crédito','nota','pbc account type']]
audit_final_30_31['débito'].fillna(0,inplace=True)
audit_final_30_31['crédito'].fillna(0,inplace=True)
audit_final_30_31.head()

In [ ]:
audit_final_30_31.shape

In [ ]:
group_poliza_30_31 = audit_final_30_31.groupby(['nota','cuenta']).sum().reset_index()
group_poliza_30_31.head()

In [ ]:
group_poliza_30_31.shape

### GROUP 01-29 JULIO 2022

In [ ]:
sheets_01_29 = pd.read_excel(url, sheet_name=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30])

In [ ]:
sheets_01_29.keys()

In [ ]:
audit_3 = pd.concat(sheets_01_29).reset_index()
audit_3.columns = audit_3.columns.str.strip().str.lower()
audit_3.head()

In [ ]:
audit_3.shape

In [ ]:
audit_3.dtypes

In [ ]:
audit_final_01_29 = audit_3[['cuenta','débito','crédito','nota','pbc account type']]
audit_final_01_29['débito'].fillna(0,inplace=True)
audit_final_01_29['crédito'].fillna(0,inplace=True)
audit_final_01_29.head()

In [ ]:
audit_final_01_29.shape

In [ ]:
group_poliza_01_29 = audit_final_01_29.groupby(['nota','cuenta']).sum().reset_index()
group_poliza_01_29.head()

In [ ]:
group_poliza_01_29.shape

# EXCEL FINAL

In [16]:
writer = pd.ExcelWriter('Revision_poliza_ns_junio_2022_vf.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
audit_final.to_excel(writer, sheet_name='Informacion_completa', index=False)
group_poliza.to_excel(writer, sheet_name='Group_all_junio', index=False)
# group_poliza_30_31.to_excel(writer, sheet_name='Group_30_31_julio', index=False)
# group_poliza_01_29.to_excel(writer, sheet_name='Group_01_29_julio', index=False)
writer.close()